In [12]:
import numpy as np
import pandas as pd
# Paquetes

# Gurobi Python Interface
from gurobipy import*

In [13]:
# Data
scenarios = [1,2,3]
hrs_week = 168
probs = {1:0.2,2:0.6,3:0.2}
demands = {1:110,2:100,3:80}
prices = {1:50,2:46,3:44}

In [14]:
# Create Model
model = Model('example 2.7')

In [15]:
# Variables
# Admission variable xi
Pc = model.addVars(scenarios,name = "Pc",lb=0,ub=90,vtype=GRB.CONTINUOUS)
# Qualification indicator
Pi = model.addVars(scenarios,name="Pi",lb=0,vtype=GRB.CONTINUOUS)

# Model update
model.update()

In [16]:
# Objective
obj = hrs_week*quicksum(probs[scenario]*(45*Pc[scenario] + prices[scenario]*Pi[scenario]) for scenario in scenarios)
model.setObjective(obj, GRB.MINIMIZE) # maximize profit

In [17]:
# Constraints
constraints = model.addConstrs(Pc[scenario] + Pi[scenario] >= demands[scenario] for scenario in scenarios)
model.addConstr(Pc[1] == Pc[2])
model.addConstr(Pc[1] == Pc[3])
model.addConstr(Pc[2] == Pc[3])

<gurobi.Constr *Awaiting Model Update*>

In [18]:
# Update model
model.update()

# Proceed to optimize
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x7a46e521
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 5e+03]
  Bounds range     [9e+01, 9e+01]
  RHS range        [8e+01, 1e+02]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4848000e+05   1.000000e+01   0.000000e+00      0s
       1    7.4793600e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.479360000e+05


In [19]:
# Visualization
rows = scenarios.copy()
columns = ["Pool","Contract"]
Pi_table = pd.DataFrame(columns=columns, index=rows, data=0.0)

for scenario in Pi.keys():
    if (abs(Pi[scenario].x) >= 0):
        Pi_table.loc[scenario, "Pool"] = np.round(Pi[scenario].x, 1)
        Pi_table.loc[scenario, "Contract"] = np.round(Pc[scenario].x, 1)
Pi_table

,Pool,Contract
1,30.0,80.0
2,20.0,80.0
3,0.0,80.0
